In [134]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import requests #--pip install requests
#!pip install pycountry
import pycountry
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split # División del dataset
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from matplotlib.cm import ScalarMappable
from matplotlib.lines import Line2D
from geopy.geocoders import Photon
from geopy.exc import GeocoderTimedOut

Retail Analysis on Large Dataset
https://www.kaggle.com/datasets/sahilprajapati143/retail-analysis-large-dataset

In [135]:
df_Retail = pd.read_csv('../data/new_retail_data.csv')

In [136]:
df_Retail.size

9060300

In [137]:
df_Retail.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,21.0,Male,Low,Regular,9/18/2023,2023.0,September,22:03:55,3.0,108.028757,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,19.0,Female,Low,Premium,12/31/2023,2023.0,December,8:42:04,2.0,403.353907,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,48.0,Male,Low,Regular,4/26/2023,2023.0,April,4:06:29,3.0,354.477600,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,56.0,Male,High,Premium,5/8/2023,2023.0,May,14:55:17,7.0,352.407717,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,22.0,Male,Low,Premium,1/10/2024,2024.0,January,16:54:07,2.0,124.276524,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies


In [138]:
df_Retail_copy=df_Retail.copy()

##Estudio univariable. Completitud, correctitud y consistencia

- Transaction_ID es el numero unico de transaccion, no deberian de haber repetidos

In [139]:
# Encontrar valores que se repiten en 'Transaction_ID'
duplicate_values_copy = df_Retail_copy[df_Retail_copy.duplicated(subset=['Transaction_ID'], keep=False)]

# Guardar duplicate_values como DataFrame
duplicate_values_copy = pd.DataFrame(duplicate_values_copy)
duplicate_values_copy.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,21.0,Male,Low,Regular,9/18/2023,2023.0,September,22:03:55,3.0,108.028757,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
32,8973612.0,65701.0,Christopher Lopez,Derrick51@gmail.com,9.286274e+09,57136 Martha Common Apt. 654,Portsmouth,England,11531.0,UK,66.0,Male,Medium,Regular,5/22/2023,2023.0,May,11:24:20,7.0,344.679185,2412.754296,Home Decor,IKEA,Decorations,Excellent,Same-Day,Debit Card,Processing,5.0,Curtains
42,3902930.0,77572.0,Jose Spencer,Amber67@gmail.com,8.282195e+09,02936 Roy Village,Portsmouth,England,98771.0,UK,68.0,Female,High,Regular,10/6/2023,2023.0,October,4:59:02,6.0,334.165698,2004.994190,Electronics,Apple,Laptop,Bad,Same-Day,Debit Card,Shipped,1.0,Microsoft Surface Laptop
72,9534452.0,20500.0,Matthew Hicks,Kara43@gmail.com,1.568422e+09,51720 Rachael Spurs Suite 283,Portsmouth,England,94447.0,UK,61.0,Female,Medium,Regular,6/6/2023,2023.0,June,8:43:39,4.0,201.953735,807.814939,Clothing,Zara,Dress,Average,Standard,PayPal,Processing,2.0,Maxi dress
93,1874788.0,74649.0,Courtney Perry,Tina32@gmail.com,7.402422e+09,571 Samuel Wall Apt. 895,Portsmouth,England,24895.0,UK,18.0,Male,High,Regular,4/19/2023,2023.0,April,22:26:07,5.0,289.116783,1445.583914,Grocery,Nestle,Chocolate,Average,Express,Debit Card,Delivered,2.0,Chocolate-covered fruits


In [140]:
df_filtered_transaction = df_Retail_copy[df_Retail_copy['Transaction_ID'] == 8691788]
df_filtered_transaction.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,21.0,Male,Low,Regular,9/18/2023,2023.0,September,22:03:55,3.0,108.028757,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
112551,8691788.0,57392.0,Jacqueline Collins,Krystal93@gmail.com,5.575568e+09,32467 Lopez Falls Suite 272,Ottawa,Ontario,88811.0,Canada,46.0,Male,Medium,New,12/29/2023,2023.0,December,20:43:59,2.0,147.379781,294.759562,Electronics,Sony,Television,Excellent,Standard,Cash,Processing,4.0,LED TV


Este resultado es inconsistente, muestra para una misma trasaccion distintos customer_id. Procedo a eliminar duplicados de Transaction_ID

In [141]:
df_Retail_copy_unique = df_Retail_copy.drop_duplicates(subset=['Transaction_ID'], keep=False)

- Customer_ID es el identificador del cliente. Se observa que para un mismo identificador de cliente, el nombre, la direccion, y los datos de la persona son distintos en las transacciones realizadas por el cliente. Esto puede ser una inconsistencia de datos o compras realizadas por familiares, amigos de la persona o robo de cuenta. Como no es posible definir cual es la situacion, continuare estudiando las otras variables y retomare luego esta inconsistencia.

In [142]:
df_Retail_copy_unique['Customer_ID'].value_counts().max()

13

Se observa que el comprador que compro mas veces lo hizo 13 veces

In [143]:
pd.set_option('display.max_columns', None)
df_filtered2 = df_Retail_copy_unique[df_Retail_copy_unique['Customer_ID'] == 10037]
df_filtered2.head(20)

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
8531,5056596.0,10037.0,Robert Wilson,James32@gmail.com,1.226992e+09,33461 Sarah Forges Suite 845,Portsmouth,England,20966.0,UK,19.0,Male,Medium,Regular,10/3/2023,2023.0,October,11:16:40,2.0,98.308952,196.617904,Grocery,Pepsi,Soft Drink,Bad,Same-Day,Cash,Delivered,1.0,Lemon-lime soda
188313,6849125.0,10037.0,Cassandra Villanueva,Meagan47@gmail.com,6.429721e+09,02641 Stewart Underpass,Adelaide,New South Wales,38930.0,Australia,44.0,Female,Low,New,4/8/2023,2023.0,April,1:33:29,3.0,302.369356,907.108068,Electronics,Apple,Tablet,Bad,Express,Debit Card,Delivered,1.0,iPad


- Name es el nombre del cliente, a efectos de corroborar el problema mencionado anteriormente, voy a dejar esta variable, pero la terminare eliminando cuando pase a la etapa de evaluacion estadistica ya que no tiene valor. Lo mismo sucede con:
- Email es el correo electronico del cliente
- Phone es el telefono del cliente
- Address direccion de envio


- City es la ciudad donde el cliente vive
Corroborar que la ciudad no tenga caracteres raros y que para una misma ciudad este escrito de forma diferente

In [144]:
df_Retail_copy_unique['City'].unique()

array(['Nottingham', 'Geelong', 'Edmonton', 'Bristol', 'Brisbane',
       'Kitchener', 'Munich', 'Wollongong', 'Cologne', 'Portsmouth',
       'San Jose', 'Hamilton', 'Manchester', 'Cardiff', 'Glasgow', 'Hull',
       'Cleveland', 'Southampton', 'Leipzig', 'Cairns', 'London',
       'Bielefeld', 'Düsseldorf', 'Philadelphia', 'Halifax', 'Montreal',
       'Dortmund', 'Mackay', 'Quebec City', 'Barrie', 'Adelaide', 'Leeds',
       'Plymouth', 'Perth', 'Sheffield', 'Frankfurt', 'Toronto',
       'Kelowna', 'Birmingham', 'Ottawa', 'Liverpool', "St. John's",
       'Hobart', 'Atlanta', 'New Orleans', 'Wichita', 'Albury-Wodonga',
       'Winnipeg', 'Vancouver', 'Hamburg', 'Windsor', 'Calgary',
       'Newcastle upon Tyne', 'Townsville', 'Oshawa', 'Houston', 'Berlin',
       'Seattle', 'Charlotte', 'New York', 'Milwaukee', 'Edinburgh',
       'Launceston', 'Bochum', 'Münster', 'Bonn', 'Columbus', 'Melbourne',
       'Leicester', 'Fort Worth', 'Toowoomba', 'Victoria', 'Oxford',
       'Saskatoo

In [145]:
df_Retail_copy_unique['Country'].unique()

array(['UK', 'Australia', 'Canada', 'Germany', 'USA', nan], dtype=object)

In [146]:
# Crear un dataframe con combinaciones únicas de ciudad y país
unique_cities = df_Retail_copy_unique[['City', 'Country']].drop_duplicates()
unique_cities = unique_cities.dropna()
print(unique_cities)

               City    Country
1        Nottingham         UK
2           Geelong  Australia
3          Edmonton     Canada
4           Bristol         UK
5          Brisbane  Australia
...             ...        ...
30592  Indianapolis        USA
30626       Detroit        USA
30635   Los Angeles        USA
30684       El Paso        USA
30981   Albuquerque        USA

[131 rows x 2 columns]


In [147]:
df_Retail_copy_unique['Country'] =df_Retail_copy_unique['Country'].replace('USA','United States')
df_Retail_copy_unique['Country'] =df_Retail_copy_unique['Country'].replace('UK','United Kingdom')

C:\Users\mariana\AppData\Local\Temp\ipykernel_10148\2480248971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Retail_copy_unique['Country'] =df_Retail_copy_unique['Country'].replace('USA','United States')
C:\Users\mariana\AppData\Local\Temp\ipykernel_10148\2480248971.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Retail_copy_unique['Country'] =df_Retail_copy_unique['Country'].replace('UK','United Kingdom')


In [148]:
def validate_country(country_name):
    try:
        # Validate by country name
       
        country = pycountry.countries.lookup(country_name)
        
        if(country_name==country.name):
           # print(country.name)
           pass
        else:
            print("No encontrado")
        return country.name
    except LookupError:
        return None

df_Retail_countries['pais_valido'] = df_Retail_countries.apply(lambda row: validate_country(row['Country']), axis=1)

In [149]:
# Filtra las filas con resultados incorrectos en la validación
df_paises_incorrectos = df_Retail_countries[df_Retail_countries['pais_valido'] == False]
if(df_paises_incorrectos.empty):
    print("Todas los paises son correctos")
else:
    print(df_paises_incorrectos)

Todas los paises son correctos


In [150]:
df_Retail_countries['pais_valido'].unique()

array(['United Kingdom', 'Australia', 'Canada', 'Germany',
       'United States', None], dtype=object)

In [151]:
df_Retail_countries=df_Retail_copy_unique.copy()

In [152]:
total_rows = df_Retail_countries.shape[0]
print(f"Total number of rows: {total_rows}")

Total number of rows: 287370


In [153]:
geolocator = Photon(user_agent="geoapiExercises")

def validate_city_country(city, country):
    try:
        # Ensure proper formatting of the query
        query = f"{city}, {country}"
        location = geolocator.geocode(query, timeout=10)
        if location and location.address:
            return location.address
        else:
            return None
    except GeocoderTimedOut:
        time.sleep(1)  # Wait a bit before retrying
        return validate_city_country(city, country)

In [154]:
print(validate_city_country('Montevideo', 'Uruguay'))

Montevideo, Montevideo, Uruguay


In [155]:
# Extract unique city-country pairs
unique_city_country_pairs = df_Retail_countries[['City', 'Country']].drop_duplicates()
unique_city_country_pairs = unique_city_country_pairs.drop_duplicates()
# Apply the validation function to the unique city-country pairs
unique_city_country_pairs['Validation'] = unique_city_country_pairs.apply(
    lambda row: validate_city_country(row['City'], row['Country']), axis=1
)

In [156]:
unique_city_country_pairs_copy=unique_city_country_pairs.copy()

In [157]:
print(f"Original dataframe shape: {df_Retail_countries.shape}")
print(f"Unique city-country pairs shape: {unique_city_country_pairs.shape}")

df_Retail_countries = df_Retail_countries.merge(unique_city_country_pairs, on=['City', 'Country'], how='left', suffixes=('', '_Validation'))

print(f"Dataframe shape after merge: {df_Retail_countries.shape}")

Original dataframe shape: (287370, 30)
Unique city-country pairs shape: (236, 3)
Dataframe shape after merge: (287370, 31)


In [158]:
# Filtra las filas con resultados incorrectos en la validación
df_incorrectas = df_Retail_countries[df_Retail_countries['Validation'].isna()]
if(df_incorrectas.empty):
    print("Todas las ciudades son correctas")
else:
    print(df_incorrectas)

Todas las ciudades son correctas


In [159]:
#for country in pycountry.countries:
#    print(country.name)

- State es el estado donde el cliente vive
- Zip code es el codigo de la direccion del cliente, este deberia ser el mismo para las mismas direcciones y deben tener el mismo largo si esta en el mismo pais y en la misma ciudad 

In [160]:
# Agrupar por 'Customer_ID' y 'Customer_Name' y contar el número de filas en cada grupo
grouped_df = df_Retail.groupby(['Customer_ID', 'Name']).size().reset_index(name='counts')

# Filtrar para encontrar los grupos con más de una compra
duplicate_purchases = grouped_df[grouped_df['counts'] > 1]

# Mostrar el resultado
print(duplicate_purchases)

        Customer_ID                     Name  counts
116         10037.0             Jessica Bell       2
159         10049.0  Mrs. Rachael Hughes DDS       2
177         10054.0          Chelsea Shelton       2
506         10150.0               Sara Lopez       2
665         10203.0          Robert Thompson       2
...             ...                      ...     ...
298508      99874.0           Madeline Henry       2
298519      99878.0              Heidi Villa       2
298679      99924.0          Justin Petersen       2
298842      99963.0          Linda Hernandez       2
298941      99993.0          Cynthia Spencer       2

[2349 rows x 3 columns]


In [161]:
df_Retail.shape

(302010, 30)

In [162]:
df_Retail.dtypes

Transaction_ID      float64
Customer_ID         float64
Name                 object
Email                object
Phone               float64
Address              object
City                 object
State                object
Zipcode             float64
Country              object
Age                 float64
Gender               object
Income               object
Customer_Segment     object
Date                 object
Year                float64
Month                object
Time                 object
Total_Purchases     float64
Amount              float64
Total_Amount        float64
Product_Category     object
Product_Brand        object
Product_Type         object
Feedback             object
Shipping_Method      object
Payment_Method       object
Order_Status         object
Ratings             float64
products             object
dtype: object

##Limpieza de datos

In [163]:
df_Retail.isna().sum().sum() # number of missing cells
round(df_Retail.isna().sum().sum() / df_Retail.size * 100, 1) # percentage of missing cells

0.1

EL 10% tiene datos faltantes

In [164]:
df_Retail.isna().sum()

Transaction_ID      333
Customer_ID         308
Name                382
Email               347
Phone               362
Address             315
City                248
State               281
Zipcode             340
Country             271
Age                 173
Gender              317
Income              290
Customer_Segment    215
Date                359
Year                350
Month               273
Time                350
Total_Purchases     361
Amount              357
Total_Amount        350
Product_Category    283
Product_Brand       281
Product_Type          0
Feedback            184
Shipping_Method     337
Payment_Method      297
Order_Status        235
Ratings             184
products              0
dtype: int64

Por mayor comodidad, transformo todas las variables a category menos las numericas

In [165]:
columns_not_to_convert = ['Age', 'Zipcode','Total_Purchases', 'Amount', 'Total_Amount', 'Ratings', 'Customer_ID', 'Transaction_ID']
all_columns = set(df_Retail.columns)

columns_not_to_convert_set = set(columns_not_to_convert)
columns_to_skip = list(all_columns - columns_not_to_convert_set)

# Convertir las columnas restantes a tipo 'category'
for col in columns_to_skip:
    df_Retail[col] = df_Retail[col].astype('category')

In [166]:
df_Retail['Age'].unique()

array([21., 19., 48., 56., 22., 58., 29., 46., 25., 64., 31., 53., 32.,
       43., 69., 49., 61., 41., 38., 59., 20., 67., 50., 26., 66., 24.,
       54., 28., 34., 65., 40., 68., 36., 57., 27., 35., 70., 37., 30.,
       39., 47., 18., 60., 33., 62., 42., 44., 51., 63., 55., 23., 52.,
       nan, 45.])

La edad esta en float, la transformo a entero

In [167]:
df_Retail = df_Retail.dropna(subset=['Age'])
df_Retail = df_Retail.dropna(subset=['Zipcode'])
df_Retail = df_Retail.dropna(subset=['Year'])

In [168]:
df_Retail['Age'] = df_Retail['Age'].astype('int')
df_Retail['Year'] = df_Retail['Year'].astype('int')
df_Retail['Zipcode'] = df_Retail['Zipcode'].astype('int')

Quiero borrar todos los valores nulos o nan pero quiero estudiar primero si me conviene borrar las filas o una columna entera, porque si los datos faltantes es tan en su mayoria en una columna puede ser mejor deshacerme de esa columna en lugar de borrar registros.
A continuacion estudio cual es la variable que tiene la mayor cantidad de nulos o valores nan

In [169]:
# Calcular el número de valores NaN por columna
nan_counts = df_Retail.isna().sum()

# Encontrar la columna con el mayor número de NaN
column_with_most_nan = nan_counts.idxmax()
max_nan_count = nan_counts.max()

# Mostrar resultados
print("La columna con la mayor cantidad de NaN es:", column_with_most_nan)
print("Número de NaN en esta columna:", max_nan_count)

La columna con la mayor cantidad de NaN es: Name
Número de NaN en esta columna: 382


Siendo que Name es la columna con la mayor cantidad de nulos y que esta no tiene valor estadistico para mi estudio, la elimino

In [170]:
df_Retail = df_Retail.drop(columns=['Name'])

In [171]:
df_Retail['Total_Amount'] = df_Retail['Total_Amount'].round(2)
df_Retail['Amount'] = df_Retail['Amount'].round(2)
df_Retail['Total_Purchases'] = df_Retail['Total_Purchases'].round(2)

In [172]:
# Quiero eliminar las filas que contengan NAN, pero no quiero quedarme con pocos datos por lo que cuento primero cuantas filas se borrarian

total_rows_before = df_Retail.shape[0]

# Eliminar todas las filas que contienen al menos un NaN en alguna columna
df_clean = df_Retail.dropna()

# Contar las filas después de eliminar NaN
total_rows_after = df_clean.shape[0]

# Calcular cuántas filas se eliminaron
rows_deleted = total_rows_before - total_rows_after

# Mostrar resultados
print("Número total de filas antes de eliminar NaN:", total_rows_before)
print("Número total de filas después de eliminar NaN:", total_rows_after)
print("Número de filas eliminadas:", rows_deleted)

Número total de filas antes de eliminar NaN: 301147
Número total de filas después de eliminar NaN: 294280
Número de filas eliminadas: 6867


Estudio la consistencia de la variable edad

In [173]:
# Verificar si hay clientes con edad menor a 18
underage_clients = (df_clean['Age'] < 18).any()

# Mostrar resultado
if underage_clients:
    print("Hay clientes cuya edad es menor a 18.")
else:
    print("No hay clientes cuya edad sea menor a 18.")

No hay clientes cuya edad sea menor a 18.


In [174]:
df_clean = df_clean.drop(columns=['Phone'])

In [175]:
df_clean['Customer_ID'] = df_clean['Customer_ID'].astype('int')
df_clean['Transaction_ID'] = df_clean['Transaction_ID'].astype('int')

In [176]:
# Contar las compras por año
compras_por_año = df_clean_unique['Year'].value_counts().sort_index()

# Mostrar el conteo de compras por año
print("Conteo de Compras por Año:")
print(compras_por_año)

NameError: name 'df_clean_unique' is not defined

Hay pocos datos para el 2024, por lo que me voy a centrar en el 2023

In [ ]:
df_clean_unique = df_clean_unique.loc[df_clean_unique['Year'] != 2024]

In [ ]:
df_clean_unique = df_clean_unique.drop(columns=['Year'])

In [ ]:
df_clean_unique['Customer_ID'] = df_clean_unique['Customer_ID'].astype('category')
df_clean_unique['Transaction_ID'] = df_clean_unique['Transaction_ID'].astype('category')

In [ ]:
df_clean_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 234312 entries, 1 to 301262
Data columns (total 27 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   Transaction_ID    234312 non-null  category
 1   Customer_ID       234312 non-null  category
 2   Email             234312 non-null  category
 3   Address           234312 non-null  category
 4   City              234312 non-null  category
 5   State             234312 non-null  category
 6   Zipcode           234312 non-null  int32   
 7   Country           234312 non-null  category
 8   Age               234312 non-null  int32   
 9   Gender            234312 non-null  category
 10  Income            234312 non-null  category
 11  Customer_Segment  234312 non-null  category
 12  Date              234312 non-null  category
 13  Month             234312 non-null  category
 14  Time              234312 non-null  category
 15  Total_Purchases   234312 non-null  float64 
 16  Amount 

##EDA

In [ ]:
df_clean_unique_customer.describe(percentiles=[.05,.5,.25,.75,.95,.99])


,Zipcode,Age,Total_Purchases,Amount,Total_Amount,Ratings
count,17394.000000,17394.000000,17394.000000,17394.000000,17394.000000,17394.000000
mean,49995.103254,35.419053,5.353110,256.214721,1374.169757,3.163217
std,29062.542596,15.086977,2.862466,141.927651,1135.256874,1.320959
min,503.000000,18.000000,1.000000,10.000000,10.000000,1.000000
5%,4793.950000,19.000000,1.000000,34.066500,115.986000,1.000000
25%,24717.500000,22.000000,3.000000,132.700000,440.347500,2.000000
50%,50380.000000,32.000000,5.000000,256.670000,1047.015000,3.000000
75%,74637.250000,46.000000,8.000000,380.570000,2036.457500,4.000000
95%,95656.150000,65.000000,10.000000,475.838500,3694.542000,5.000000
99%,99200.030000,70.000000,10.000000,494.945600,4537.283200,5.000000


La edad no la considero por que los customers estan repetidos, por lo que debo primero hacer una limpieza y luego estudiar estadisticamente los datos (lo hare a continuacion)
Total_Purchases -- Es el numero de compras realizadas por el customer
En promedio se compran 5 articulos por compra. Siendo 1 el minimo, y 10 el maximo. En esta variable no se observan valores atipicos ni la necesidad de realizar una estandarizacion. 
En promedio los articulos cuestan 

Como las personas se repiten, si quiero estudiar estadisticamente la edad de estas, tengo que hacer un drop de duplicados de Customer_ID. Commo todos los datos son en el mismo a;o, la edad de la persona va a ser la misma para todas sus compras, por lo que no me preocupa cual registro me quedo

In [ ]:
df_clean_unique_customer=df_clean_unique.copy()
df_clean_unique_customer = df_clean_unique_customer.drop_duplicates(subset=['Customer_ID'], keep=False)